In [1]:
#所需要的库
#The required libraries
import re
from bs4 import BeautifulSoup
from concurrent import futures
import sys
import time
import pandas as pd
import requests
import json
from fake_useragent import UserAgent
import urllib3.contrib.pyopenssl
import urllib3
import logging


# Set the base parameters
urllib3.disable_warnings()
logging.captureWarnings(True)
urllib3.contrib.pyopenssl.inject_into_urllib3()
HEADERS = {}
NUM_THREADS = 5
PRINT = False
proxies = None
ua = UserAgent()

class JobSpider(object):

    def __init__(self):
        self.headers = {"User-Agent": UserAgent().random,'Connection':'close'}
        self.datas = list()
        self.countforall = 0
        self.dataset=pd.DataFrame(columns=["职位名称","薪资","所在地", "职位简介","职位详情",
                                     "经验要求", "学历要求", "招聘人数",
                                     "公司名称", "类型","规模",  "领域", "公司详情"])
    #获取最大可爬取页数
    #Gets the maximum number of pages to climb
    
    def getMaxPage(self, url):
        response = requests.get(url, headers = self.headers)
        if response.status_code == 200:
            source = response.text
            soup = BeautifulSoup(source, "html.parser")
            maxPage = soup.find("input", {"id" : "hidTotalPage"})["value"]
            return  int(maxPage)
        else:
            print("Fail status: {}".format(response.status_code))
            return None
        
    def getDetailPages(self,url):
        job=[]
        company=[]
        count=0
        print("当前正在爬取: {}".format(url))
        response = requests.get(url, headers = self.headers)
        soup = BeautifulSoup(response.text, "html.parser")
        link_company = soup.find_all("div", class_ = "el")
        link_job = soup.find_all("div", class_ = "el")
        for i in link_company:
            link_c1 = i.find_all("span", class_="t2")
            for i in link_c1:
                try:
                    link_c2=i.find("a")["href"]
                    company.append(link_c2)
                except Exception as e:
                    print("爬取公司链接时", e)
        for i in link_job:
            link_j1 = i.find_all("p", class_="t1")
            for i in link_j1:
                try:
                    link_j2 = i.find("a")["href"]
                    job.append(link_j2)
                except Exception as e:
                    print("爬取工作链接时", e)
        return job,company
        
    #连接和数据记录函数 
    #Connection and data recording functions
    def parsePage(self, url):
        maxPage = self.getMaxPage(url)
        for pageNum in range(180, maxPage+1):
            url = "https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,{}.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=".format(pageNum)
            print("当前正在爬取: 第{}页".format(pageNum))
            joburl,comurl=self.getDetailPages(url)
            with futures.ThreadPoolExecutor(max_workers =10) as t:
                obj_list = []
                for i,j in zip(joburl,comurl):
                    obj = t.submit(self.detail,i,j)
                    obj_list.append(obj)
                for future in futures.as_completed(obj_list):
                    data = future.result()
                    print(f"main: {data}")
        self.dataset.to_csv("Jobs.csv",encoding='utf_8_sig', index=False)


    #数据获取函数    
    #Data acquisition function
    def detail(self,url1,url2):
        try: 
            r_job = requests.get(url1, headers = self.headers,verify=False)
            r_job.encoding = "gbk"
            r_com = requests.get(url2, headers = self.headers,verify=False)
            r_com.encoding = "gbk"
            bs_obj_com = BeautifulSoup(r_com.text, "html.parser")
            bs_obj_job = BeautifulSoup(r_job.text, "html.parser")
            if bs_obj_job is not None:
                try:
                    div_job = bs_obj_job.find("div", class_="cn")
                    div_title_job = div_job.find("h1")["title"]
                    div_salary_job =div_job.find("strong").get_text()
                    div_info_job = "".join(div_job.find("p", class_="msg ltype")["title"].split())
                    title_strs_job = div_info_job.split("|")
                    if "发布" in title_strs_job[4]:
                        loc_1_job = title_strs_job[0].split("-")[0]
                        exp_job = title_strs_job[1]
                        edu_job = title_strs_job[2]
                        num_job = title_strs_job[3].replace("招","").replace("人","")
                        div_tag_job = div_job.find_all("span", class_="sp4")
                        tag_job =""
                        for i in div_tag_job:
                            tag_job = tag_job+"|"+i.get_text()
                        div2_job = bs_obj_job.find("div", class_="bmsg job_msg inbox")
                        div_detail_job = div2_job.find_all("p")
                        detail_job=""
                        for i in div_detail_job:
                            detail_job=detail_job+"|"+i.get_text()
                        div_com = bs_obj_com.find("div", class_="tHeader tHCop")
                        div_title_com = div_com.find("h1")["title"]
                        div_info_com = "".join(div_com.find("p", class_="ltype")["title"].split())
                        title_strs_com = div_info_com.split("|")
                        type_com = title_strs_com[0]
                        staff = title_strs_com[1]
                        field_com = title_strs_com[2]
                        div2_com = bs_obj_com.find("div", class_="con_txt")
                        detail_com = "".join(div2_com.get_text().split())
                        temp_df = pd.DataFrame(data=[[div_title_job,div_salary_job,loc_1_job,tag_job,detail_job,
                                              exp_job, edu_job, num_job,div_title_com, type_com, staff, field_com, 
                                              detail_com]], 
                                    columns=["职位名称","薪资","所在地", "职位简介","职位详情",
                                             "经验要求", "学历要求", "招聘人数",
                                             "公司名称", "类型","规模",  "领域", "公司详情"])
                        self.dataset = self.dataset.append(temp_df)
                        self.countforall +=1
                        self.dataset.to_csv("Jobs.csv",encoding='utf_8_sig', index=False)
                        print("已爬取信息：", self.countforall)
                except Exception as e:
                    print(e)
        except Exception as e:
            print(e)
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            time.sleep(5)
            print("Was a nice sleep, now let me continue...")
        return None
    
        
if __name__ == "__main__":
    Jobs = JobSpider()
    Jobs.parsePage("https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,1.html?lang=c&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&ord_field=0&dibiaoid=0&line=&welfare=")

当前正在爬取: 第180页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,180.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
已爬取信息： 10
main: None
已爬取信息：已爬取信息： 10
 10
已爬取信息：main: None 10

main: None
main: None
已爬取信息： 10
main: None
已爬取信息： 10
main: None
已爬取信息： 10
main: None
已爬取信息： 10
main: None已爬取信息： 10

main: None
已爬取信息： 10
main: None
已爬取信息： 20
已爬取信息： 20
main: None
main: None
已爬取信息： 20
已爬取信息：main: None
 20
main: None
已爬取信息： 20
已爬取信息： 20
main: None
main: None
已爬取信息： 20
已爬取信息：main: None
 20
main: None已爬取信息：
 20
main: None
已爬取信息： 20
main: None
list index out of range
main: None
已爬取信息： 29
已爬取信息：main: None 29

main: None
已爬取信息： 29
main: None
已爬取信息： 29
main: None
已爬取信息： 29
main: None
已爬取信息： 29
main: N

爬取公司链接时 'NoneType' object is not subscriptable
main: None
已爬取信息： 226
main: None
已爬取信息： 226
main: None
已爬取信息： 226
已爬取信息： main: None
226
main: None
已爬取信息： 已爬取信息： 226
226main: None

main: None
已爬取信息： 226
main: None
已爬取信息： 226
main: None已爬取信息： 226

main: None
list index out of range
main: None
已爬取信息： 235
main: None
已爬取信息： 235
main: None
已爬取信息： 235
main: None
已爬取信息： 已爬取信息：235 235

main: None
main: None
已爬取信息： 235
main: None
已爬取信息： 235
main: None已爬取信息：
 235
main: None
已爬取信息： 235
main: None
list index out of range
main: Nonelist index out of range

list index out of range
main: None
main: None已爬取信息： 242

main: None
已爬取信息： 242
main: None
已爬取信息： 242
已爬取信息： 242
main: None
main: None
已爬取信息： 242
main: None已爬取信息：
 242
main: None已爬取信息： 
242
main: None
list index out of range
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
已爬取信息： 251
main: None
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 261
main: No

已爬取信息： 366
main: None
list index out of range
main: None
已爬取信息： 374
main: None
已爬取信息： 已爬取信息： 374
main: None
374
已爬取信息：main: None 374

main: None
已爬取信息： 374
main: None
已爬取信息： 374已爬取信息：
main: None 374

已爬取信息： 374
main: None
main: None
当前正在爬取: 第189页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,189.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
main: None
已爬取信息： 381
main: None
已爬取信息： 383
main: None
已爬取信息： 383
main: None已爬取信息：
 383
已爬取信息： 383
main: None
main: None已爬取信息：
 383
已爬取信息：main: None 383

main: None
已爬取信息： 383
main: None已爬取信息： 383

main: None
已爬取信息： 384
main: None
list index out of rangelist index out of range

main: None
main: None
已爬取信息： 391
main: No

爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
list index out of rangemain: None

main: None
list index out of range
main: None
已爬取信息： 593
main: None
已爬取信息：已爬取信息： 593
 593
main: None
main: None
已爬取信息： 593
main: None
已爬取信息： 593
main: None
已爬取信息： 593
main: None
已爬取信息： 593
main: None
list index out of range
main: None
list index out of range
main: None
已爬取信息： 601已爬取信息： 601
main: None

已爬取信息：main: None 601

main: None
已爬取信息： 601
main: None
已爬取信息： 601已爬取信息： 601

main: None
main: None
已爬取信息： 已爬取信息： 601
main: None
601
main: None
list index out of range
main: None
list index out of range
main: None
已爬取信息： 609
main: None
已爬取信息： 609
main: None
已爬取信息： 609
main: None已爬取信息：已爬取信息： 609
 
609main: None

main: None
已爬取信息：已爬取信息： 609
 609
main: None
main: None
已爬取信息： 609
main: None
list index out of range
main: None
已爬取信息： 618
已爬取信息： 618
main: None
main: None
已爬取信息： 618
main: None
已爬取信息：已爬取信息： 618
main: None
 618
已爬取信息： 618
main: None
main: None
已爬取信息：已爬取信息： 618
 main: None618

ma

已爬取信息：已爬取信息： 812
main: None 
812
main: None
已爬取信息： 812
main: None
已爬取信息： 812
main: None
已爬取信息： 812
main: None
已爬取信息：已爬取信息： 812
 main: None
812
main: None
已爬取信息： 812
已爬取信息： 812已爬取信息： 
812
main: None
main: None
main: None
list index out of rangelist index out of range

main: None
list index out of rangemain: None

main: None
已爬取信息： 819
main: None
已爬取信息： 819
main: None
已爬取信息： 819
main: None
已爬取信息： 819
main: None
已爬取信息： 819
已爬取信息：main: None
 819
main: None
已爬取信息： 819
main: None
已爬取信息： 820
main: None
list index out of range
list index out of rangemain: None

main: None
已爬取信息：已爬取信息：  828
main: None828

main: None
已爬取信息： 828
main: None
已爬取信息：已爬取信息： 828
 main: None
828
main: None
已爬取信息： 已爬取信息： 828
828main: None

main: None
已爬取信息： 828
main: None
list index out of range
main: None
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
当前正在爬取: 第200页
当前正在爬取: https://search.51job.com/list/0

...

爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
main: None
已爬取信息： 3045
已爬取信息：main: None 3045

main: None
已爬取信息： 3045已爬取信息： 3045

main: None
main: None
已爬取信息： 已爬取信息：3045
 main: None3045

main: None
已爬取信息： 3045已爬取信息：
 3045
main: None
main: None
已爬取信息： 3045
main: None
已爬取信息： 3045
main: None
list index out of range
main: None
已爬取信息： 3055
main: None
已爬取信息： 3055
main: None
已爬取信息： 已爬取信息：3055 
已爬取信息： 3055
main: None3055
main: None

main: None
已爬取信息： 3055
main: None
已爬取信息： 已爬取信息：3055
main: None 3055

main: None
已爬取信息： 3055
main: None
已爬取信息： 3055
main: None
list index out of range
main: None
已爬取信息： 3065
main: None已爬取信息：
 3065
main: None
已爬取信息： 3065
main: None
已爬取信息：已爬取信息：  3065
已爬取信息： 3065
main: None
main: None
3065
main: None
已爬取信息： 3065
main: None
已爬取信息： 3065
main: None
已爬取信息： 3065
main: None
已爬取信息： 3065
main: None
list index out of range
main: None
已爬取信息： 3075
main: None
已爬取信息： 3075
main: None已爬取信息：
 3075
main: None
已爬取信息： 3075
main: None
已爬取信息： 3075
main: None
已爬取信息： 3

Was a nice sleep, now let me continue...
main: None
当前正在爬取: 第259页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,259.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
main: None
list index out of range
list index out of rangemain: None

main: None
main: None
已爬取信息： 3255
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息：已爬取信息： 3256 
3256main: None

main: None
list index out of range
main: None
已爬取信息： 3256
main: None
已爬取信息： 3259
list index out of range
main: None
main: None
已爬取信息：已爬取信息：  3260已爬取信息： 
3260main: None3260


main: None
main: None
已爬取信息： 3260
main: None
list index out of

HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries exceeded with url: /all/co404548.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002A766FC5648>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))
Connection refused by the server..
Let me sleep for 5 seconds
Was a nice sleep, now let me continue...
main: None
Was a nice sleep, now let me continue...
main: None
Was a nice sleep, now let me continue...Was a nice sleep, now let me continue...

main: None
main: None
已爬取信息： 3335
main: None
已爬取信息： 3335
main: None
已爬取信息：已爬取信息： 3335
 3335main: None

main: None
Was a nice sleep, now let me continue...
main: None
Was a nice sleep, now let me continue...
main: None
main: None
Was a nice sleep, now let me continue...
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
list index out of range
main:

Was a nice sleep, now let me continue...
main: None
当前正在爬取: 第266页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,266.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
list index out of rangemain: None

main: Nonelist index out of range

main: None
已爬取信息： 3526
main: None
已爬取信息：已爬取信息：  35263526
main: None

main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
list index out of range
main: None
main: None
已爬取信息： 3535
main: None
已爬取信息：已爬取信息：已爬取信息： 3535
 main: None3535
 
3535
main: None
main: None
已爬取信息： 3535已爬取信息： 3535

main: None
main: None
已爬取信息：

ConnectionError: HTTPSConnectionPool(host='search.51job.com', port=443): Max retries exceeded with url: /list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,270.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare= (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002A765697EC8>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [46]:
data = pd.read_csv("Jobs.csv", nrows=1500)

In [9]:
data.head()

,职位名称,薪资,所在地,职位简介,职位详情,经验要求,学历要求,招聘人数,公司名称,类型,规模,领域,公司详情
0,数据分析员,0.8-1万/月,佛山,NaN,|职能类别：大数据开发工程师|关键字：数据分析电商数据,1年经验,本科,1,美的集团热水器事业部,民营公司,5000-10000人,家具/家电/玩具/礼品,美的厨热事业部是美的集团旗下专业研发、制造和营销电热水器、燃气热水器的供应商，拥有目前全球最...
1,销售数据分析,0.8-1万/月,上海,|五险一金|年终奖金|定期体检|带薪年假|免费班车|节日福利|专业培训|补充医疗保险|员工旅...,|1. 根据销售部门的业务发展要求，提供销售分析数据，包括但不限于|a. 每...,1年经验,本科,若干,安硕文教用品（上海）股份有限公司,合资,1000-5000人,办公用品及设备,安硕文教用品（上海）股份有限公司是外商投资股份制企业，是全球最顶尖的木质铅笔生产企业之一；也...
2,电商AI系统数据分析师,2-2.5万/月,上海,NaN,|背景： |百胜中国是中国***的餐饮集团，拥有肯德基、必胜客等著名品牌|百胜中国的数字化建...,3-4年经验,本科,1,百胜咨询（上海）有限公司,外资（欧美）,500-1000人,餐饮业,"百胜中国控股有限公司（YumChinaHoldings,Inc.）是中国领先的餐饮公司，致力..."
3,数据分析专员,0.8-1万/月,广州,|五险一金|餐饮补贴|绩效奖金|年终奖金|定期体检|电脑补贴|员工旅游|专业培训|交通补贴,|岗位职责：|1、日常监控数据，实时发现并解决广告投放中出现的问题；|2、负责部门广告数据整...,1年经验,本科,4,三七互娱,上市公司,1000-5000人,网络游戏,三七互娱，全球TOP25上市游戏企业，位列中国第三，凭借优异业绩被纳入中证沪深300指数、明...
4,薪酬数据分析,NaN,上海,NaN,|1.参加薪酬福利市场调研与数据分析；2.参加薪酬福利年度项目工作的开展：年度调薪、奖金方案...,3-4年经验,硕士,1,中芯国际集成电路制造（上海）有限公司,外资（非欧美）,10000人以上,电子技术/半导体/集成电路,中芯国际集成电路制造有限公司（“中芯国际”，纽交所代号：SMI，港交所股份代号：981），是...


In [54]:
data=data.dropna(axis=0,how='any')

In [57]:
def xinzi(xinzi):
    if "万" in xinzi:
        if "月" in xinzi: 
            if "-" in xinzi:
                s = re.split('[-万]',xinzi)
                s1,s2 = float(s[0]),float(s[1])
                salary = ((s1+s2)/2)*10000
                return round(salary,0)
            else:
                s = re.split('[万]',xinzi)
                salary = float(s[0])*10000
                return salary               
        if "年" in xinzi:
            if "-" in xinzi:
                s = re.split('[-万]',xinzi)
                s1,s2=float(s[0]),float(s[1])
                salary = ((s1+s2)/2)*10000/12
                return round(salary,0)
            else:
                s = re.split('[万]',xinzi)
                salary = float(s[0])*10000/12
                return round(salary,2) 
    if "千" in xinzi:
        if "-" in xinzi:
            s = re.split('[-千]',xinzi)
            s1,s2 = float(s[0]),float(s[1])
            salary = ((s1+s2)/2)*1000
            return round(salary,0)
        else:
            s = re.split('[千]',xinzi)
            salary = float(s[0])*1000
            return salary  
    if "天" in xinzi:
        return float(0)

def exp(exp):
    if "在校生" in exp or "应届生" in exp or "无需经验" in exp:
        return float(0)
    if "年" in exp:
        if "-" in exp:
            e = re.split('[-年]',exp)
            e1,e2 = float(e[0]),float(e[1])
            ex = ((e1+e2)/2)
            return round(ex,1)
        else:
            e = re.split('[年]',exp)
            ex = float(e[0])
            return ex  
        
def guimo(guimo):
    if "-" in guimo:
        s = re.split('[-人]',guimo)
        s1,s2 = float(s[0]),float(s[1])
        scale = ((s1+s2)/2)
        return round(scale,1)
    if "少于" in guimo:
        return(float(50))
    if "以上" in guimo:
        return(float(10000))

In [56]:
salary=[]
for i in data["薪资"]:
    a = xinzi(i)
    salary.append(a)
data.drop(columns=["薪资"])
data["薪资"]=salary

ex=[]
for i in data["经验要求"]:
    a = exp(i)
    ex.append(a)
data.drop(columns=["经验要求"])
data["经验要求"]=ex

scale=[]
for i in data["规模"]:
    a = guimo(i)
    scale.append(a)
data.drop(columns=["规模"])
data["规模"]=scale
data

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,职位名称,薪资,所在地,职位简介,职位详情,经验要求,学历要求,招聘人数,公司名称,类型,规模,领域,公司详情
1,销售数据分析,9000.0,上海,|五险一金|年终奖金|定期体检|带薪年假|免费班车|节日福利|专业培训|补充医疗保险|员工旅...,|1. 根据销售部门的业务发展要求，提供销售分析数据，包括但不限于|a. 每...,1.0,本科,若干,安硕文教用品（上海）股份有限公司,合资,1000-5000人,办公用品及设备,安硕文教用品（上海）股份有限公司是外商投资股份制企业，是全球最顶尖的木质铅笔生产企业之一；也...
3,数据分析专员,9000.0,广州,|五险一金|餐饮补贴|绩效奖金|年终奖金|定期体检|电脑补贴|员工旅游|专业培训|交通补贴,|岗位职责：|1、日常监控数据，实时发现并解决广告投放中出现的问题；|2、负责部门广告数据整...,1.0,本科,4,三七互娱,上市公司,1000-5000人,网络游戏,三七互娱，全球TOP25上市游戏企业，位列中国第三，凭借优异业绩被纳入中证沪深300指数、明...
5,数据分析师,25000.0,上海,|五险一金|补充医疗保险|免费班车|餐饮补贴|年终奖金,|职能类别：数据分析师,3.5,本科,1,完美世界（上海）/完美时空,民营公司,150-500人,网络游戏,完美世界(北京)网络技术有限公司（以下简称：完美世界）是中国领先的网络游戏开发商和运营商之一...
8,数据分析师,7000.0,南通,|五险一金|弹性工作|专业培训|员工旅游,|1、时间相对自由；|2、高收入；|3、高挑战；|4、自身素质的全面发展；|岗位职责：|1、...,0.0,大专,5,南通普思商务信息咨询有限公司,民营公司,50-150人,金融/投资/证券,无锡元坤资产管理有限公司成立于2015年旗下设有元坤金融学院（YuanKunCollegeo...
9,数据分析师,21500.0,上海,|五险一金|绩效奖金|节日福利|定期团建|周末双休,|1、基于大数据平台，提取、处理业务数据，通过 数据分析和数据挖掘，给出直观的结论及业务洞见...,6.0,本科,若干,上海灿谷投资管理咨询服务有限公司,上市公司,1000-5000人,金融/投资/证券,灿谷（纽约证券交易所股票代码：CANG）成立于2010年，由一批最早在国内从事科技化汽车金融...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,销售助理,6500.0,上海,|五险一金|周末双休|定期体检|绩效奖金,|1、收集销售预测数据、账款数据，制作各类销售支持的数据报告，销售分析，撰写销售分析报告；做...,1.0,本科,5,上海法赫桥梁隧道养护工程技术有限公司,民营公司,50-150人,计算机软件,法沃是国内一家工程行业技术解决方案提供商，提供公共交通建设软件平台及解决方案，以满足用户信息...
1496,农业规划工程师,12500.0,广州,|五险一金|补充医疗保险|补充公积金|员工旅游|专业培训|绩效奖金|年终奖金|弹性工作,|一、职位要求：|1）硕士以上学历，如有3年以上丰富项目经验者可考虑本科：|2）农业类、农产...,0.0,硕士,10,广东丰澍工程咨询有限公司,民营公司,少于50人,农/林/牧/渔,招聘启事一、公司简介：广东丰澍工程咨询有限公司在国家科技体制改革的背景下，由多名行业内的知名...
1497,销售助理（无业绩要求）,9000.0,上海,|通讯补贴|专业培训|弹性工作|餐饮补贴|帅哥美女成群|扁平化管理|老板nice,|1.台账管理：合同及相关信息整理，登记并维护台账，定期归纳提炼台账情况集中汇报；|2.数据...,3.5,本科,2,上海费盟企业管理有限公司,民营公司,少于50人,服装/纺织/皮革,深思熟虑Thoughtful叛逆革新Rebellious无所不知In-the-know虚怀若...
1498,电商运营经理,8500.0,青岛,|五险一金|商业保险|带薪年假|各项补贴|出国机会|专业培训|高温补贴|定期体检|节日福利|...,||Job Responsibilities|||1、独立运营淘宝/拼多多店铺，包括店铺定位...,1.0,本科,1,联润翔（青岛）纺织科技有限公司,民营公司,150-500人,贸易/进出口,【集团介绍】Lerune集团成立于2003年，母公司为联润翔（青岛）纺织科技有限公司，旗下有...


In [63]:
data = data[~data['薪资'].isin([0])]
data.to_csv("Jobs_clean.csv",encoding='utf_8_sig', index=False)

In [66]:
data['学历要求'].unique()

array(['本科', '大专', '硕士', '高中', '中专', '博士', '中技'], dtype=object)

In [78]:
data[data['经验要求'].isin([2])]

,职位名称,薪资,所在地,职位简介,职位详情,经验要求,学历要求,招聘人数,公司名称,类型,规模,领域,公司详情
14,数据分析师,12500.0,上海,|五险一金|餐饮补贴|交通补贴|弹性工作|员工旅游|绩效奖金|年终奖金|员工商业保险,|岗位职责：| 一、日常工作 1、负责录入数据库信息的清洗包括整理、检核、补充等，确保数...,2.0,本科,若干,上海魔拾科技有限公司,民营公司,50.0,计算机软件,上海魔拾科技有限公司AmorosTech成立于2018年，专注于用领先的互联网技术颠覆传统零...
17,数据分析师,9500.0,合肥,|五险一金|周末双休|餐饮补贴|员工旅游|定期体检|绩效奖金|年终奖金|股票期权,|岗位职责：|1、负责大气监测站多仪器的数据联合分析；|2、提交数据分析报告及数据联合应用研...,2.0,硕士,2,合肥光博量子科技有限公司,民营公司,100.0,电子技术/半导体/集成电路,合肥光博量子科技有限公司是国家高新技术企业，是中科院合肥技术创新工程院重点扶持企业。公司坚持...
18,数据分析师,17500.0,上海,|五险一金|股票期权|做五休二|带薪年假|绩效奖金|专业培训|员工旅游,|1、理解公司内部或客户的洞察需求，利用结构化和非结构化的大数据进行消费者洞察分析；|2、有...,2.0,硕士,1,飞诊健康科技（上海）有限公司,民营公司,100.0,互联网/电子商务,飞诊健康科技（上海）有限公司（简称“飞诊”）是一家互联网+医疗实体连锁的新型医疗科技企业。公...
20,销售业务-数据分析岗,5250.0,烟台,|带薪年假|五险一金|交通补贴|高温补贴|餐饮补贴|节日福利|专业培训|绩效奖金|加班补贴,|1、深入理解集团相关业务的方向和需求，应用先进的统计建模、数据挖掘、机器学习等方法建立数据...,2.0,本科,2,益海(烟台)粮油工业有限公司,外资（非欧美）,325.0,快速消费品(食品、饮料、化妆品),益海嘉里简介益海嘉里集团，是以益海嘉里金龙鱼粮油食品股份有限公司为核心的企业集团的简称，是著...
22,临床数据分析专员,12500.0,广州,|绩效奖金|餐饮补贴|员工旅游|专业培训|股票期权|定期体检|项目奖金|做五休二|交通便利|...,||岗位职责|1、根据客户需求制定数据挖掘、统计分析解决方案，参与实施过程中算法研究和实现，...,2.0,本科,若干,广州天鹏计算机科技有限公司,民营公司,325.0,计算机软件,公司介绍广州天鹏计算机科技有限公司成立于2007年，是一家专注于建设医疗专科大数据分析和管理...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,信息流优化师,8000.0,珠海,|五险一金|员工旅游|专业培训|年终奖金|周末双休,|岗位职责： 1、负责日常信息流平台（如今日头条、粉丝通、广点通、智汇推等）等渠道投放的操作...,2.0,大专,1,珠海宣传易网络科技有限公司,民营公司,100.0,互联网/电子商务,宣传易传媒成立于2001年，多年来，一直专注于互联网广告的策划、推广、营销服务。利用自主研发...
1484,百度竞价专员,7000.0,广州,|五险一金|交通补贴|通讯补贴|专业培训|年终奖金|弹性工作,|工作职责：|1．结合店铺运营计划，制定并执行直通车/钻展/淘宝客投放方案，提高店铺点击率、...,2.0,大专,2,广东美捷威通生物科技有限公司,民营公司,325.0,医疗设备/器械,"广东美捷威通生物科技有限公司成立于2011年12月,注册资本为2300万元。是一家集研发、生..."
1489,渠道商务专员,7000.0,上海,|五险一金|员工旅游|交通补贴|餐饮补贴|通讯补贴|绩效奖金|定期体检|年终奖金,|1、协助领导完成各类商务谈判；|2、配合领导完成市场规划数据分析，为品牌开撤店提供决策依据...,2.0,本科,1,天创时尚-沪浙区,上市公司,325.0,批发/零售,天创时尚股份有限公司，国内时尚女鞋行业领先的多品牌运营商之一。1998年创立，产品涵盖时尚领...
1493,销售业务支持,5500.0,上海,|五险一金|周末双休|定期体检|绩效奖金|专业培训,|1、收集销售预测数据、账款数据，制作各类销售支持的数据报告，销售分析，撰写销售分析报告；做...,2.0,本科,若干,上海法赫桥梁隧道养护工程技术有限公司,民营公司,50.0,建筑/建材/工程,上海法赫铁道技术设备有限公司，2004年04月28日成立，经营范围包括清洗设备、通讯器材、机...


In [1]:
import pandas as pd
data = pd.read_csv("Jobs_clean.csv")

In [2]:
data.to_excel("data.xlsx",index = False)

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://www.pingan.com中国平安在2017年《福布斯》“全球上市公司2000强”中名列第16位，居全球多元化保险企业***；美国《财富》世界500强第39位，并蝉联中国内地混合所有制企业***；除此之外，在英国WPP集团旗下MillwardBrown公布的2017“BrandZTM最具价值中国品牌100强”及“全球品牌100强”中，分別排名第8位及第61位；在BrandFinance发布的“2017年全球最具价值保险品牌100强”排行榜中，荣膺全球保险品牌***位。在全球***品牌咨询公司Interbrand发布的“2017年***中国品牌排行榜”中，名列第六位，并蝉联中国保险业***。' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'http://www.ricoh.com.cn理光（中国）有限公司是日本株式会社理光为进一步发展中国市场、统筹管理理光在中国大陆的营销工作而设立的全资子公司，列名于世界财富500强。理光（中国）依托日本理光作为首家「OA设备制造厂商」所积累的70年全球领先的研发和管理经验，向用户提供强有力的商务活动解决方案和周到细致的服务。秉承“始终站在客户的立场上思考和行动”的方针，使理光在全世界范围内赢得高度评价。理光集团在不断提高业务水平的同时，还积极承担环境保护和贡献于社会的企业责任。理光多元化的企业文化和广纳良才的包容力为员工提供了开放的发展空间，理光（中国）真诚期待志同道合的优秀专业人才为我们带来更多的智慧、热情和创造力，共同创造美好未来。' with link or location/anchor > 255 characters since it exceeds

In [134]:
import pandas as pd
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer
data = pd.read_excel("data.xlsx")

In [135]:
# Split keywords
def get_keywords(word):
    keywords = jieba.analyse.extract_tags(word)
    keywords = '|'.join(keywords)
    return keywords

In [136]:
aim = ['职位简介','职位详情','公司详情','领域']

In [137]:
for i in aim:
    data['keywords_{}'.format(i)]=[get_keywords(str(word)) for word in data['{}'.format(i)]]
    vectorizer = CountVectorizer(max_features=50,analyzer='word',tokenizer=lambda s:s.split('|'))
    bow = vectorizer.fit_transform(data['keywords_{}'.format(i)])
    for z in range(50):
        a=[]
        for j in bow:
            a.append(j.toarray().tolist()[0][z])
        data['{}_{}'.format(i,z)] = a

In [138]:
data.to_excel("data2.xlsx",index = False)

In [139]:
from collections import Counter
for i in aim:
    cou=[]
    for j in data['{}'.format(i)]:
        keywords = jieba.analyse.extract_tags(str(j))
        c = Counter()
        cou.extend(keywords)
    c = Counter(cou)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(50)
    a = pd.DataFrame({'word':wc_data['word'],'counts':wc_data['counts']})
    a.to_excel("keywords_of_{}.xlsx".format(i),index = False)